# 📊 Dashboard de Maturité Digitale des Cliniques

## Analyse Complète et Système de Scoring

In [1]:
# Import des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print('✅ Bibliothèques chargées avec succès!')

✅ Bibliothèques chargées avec succès!


## 1. Chargement et Préparation des Données

In [2]:
# Chargement des données
df = pd.read_csv('../../donnees_cliniques_nettoyees.csv')

print(f"📊 Dataset : {df.shape[0]} cliniques × {df.shape[1]} variables")
print(f"\nColonnes clés pour la maturité digitale:")
key_cols = ['clinic_name', 'location', 'number_of_beds', 'has_informatic_management_system',
            'uses_website', 'uses_social_media', 'uses_digital_tools_for_appointments']
print([col for col in key_cols if col in df.columns])

📊 Dataset : 28 cliniques × 129 variables

Colonnes clés pour la maturité digitale:
['clinic_name', 'location', 'number_of_beds', 'has_informatic_management_system', 'uses_website', 'uses_social_media', 'uses_digital_tools_for_appointments']


In [3]:
# Conversion des colonnes binaires
def convert_to_binary(value):
    """Convertit les réponses textuelles en valeurs binaires"""
    if pd.isna(value):
        return 0
    value_str = str(value).lower().strip()
    
    # Valeurs positives
    if value_str in ['oui', 'yes', '1', 'true']:
        return 1
    # Valeurs partielles
    elif 'partiellement' in value_str or 'mixte' in value_str:
        return 0.5
    elif 'logiciel' in value_str and 'dédié' in value_str:
        return 1
    elif 'équipe' in value_str:
        return 1
    # Valeurs négatives
    elif value_str in ['non', 'no', '0', 'false'] or 'pas' in value_str or 'manuelle' in value_str:
        return 0
    else:
        return 0

# Application aux colonnes pertinentes
binary_cols = [
    'has_informatic_management_system',
    'uses_website',
    'uses_social_media',
    'uses_digital_tools_for_appointments',
    'has_formal_digital_strategy',
    'has_dedicated_digital_team',
    'has_dedicated_digital_budget'
]

for col in binary_cols:
    if col in df.columns:
        df[col] = df[col].apply(convert_to_binary)
        print(f"✅ {col}: {df[col].sum()} cliniques (sur {len(df)})")
    else:
        df[col] = 0
        print(f"⚠️  {col}: colonne non trouvée, initialisée à 0")

✅ has_informatic_management_system: 22 cliniques (sur 28)
✅ uses_website: 0 cliniques (sur 28)
✅ uses_social_media: 0 cliniques (sur 28)
✅ uses_digital_tools_for_appointments: 8.5 cliniques (sur 28)
✅ has_formal_digital_strategy: 9 cliniques (sur 28)
✅ has_dedicated_digital_team: 6 cliniques (sur 28)
✅ has_dedicated_digital_budget: 4 cliniques (sur 28)


## 2. Calcul du Score de Maturité Digitale

In [4]:
# Pondération des indicateurs (total = 100 points)
weights = {
    'has_informatic_management_system': 25,
    'uses_website': 20,
    'uses_social_media': 20,
    'uses_digital_tools_for_appointments': 15,
    'has_formal_digital_strategy': 10,
    'has_dedicated_digital_team': 5,
    'has_dedicated_digital_budget': 5
}

# Calcul du score pour chaque clinique
df['maturity_score'] = 0
for col, weight in weights.items():
    df['maturity_score'] += df[col] * weight

# Arrondir à 1 décimale
df['maturity_score'] = df['maturity_score'].round(1)

print("📊 Statistiques du Score de Maturité Digitale:")
print(f"\nMoyenne: {df['maturity_score'].mean():.1f}/100")
print(f"Médiane: {df['maturity_score'].median():.1f}/100")
print(f"Min: {df['maturity_score'].min():.1f}/100")
print(f"Max: {df['maturity_score'].max():.1f}/100")
print(f"Écart-type: {df['maturity_score'].std():.1f}")

📊 Statistiques du Score de Maturité Digitale:

Moyenne: 29.2/100
Médiane: 32.5/100
Min: 0.0/100
Max: 55.0/100
Écart-type: 17.7


In [5]:
# Catégorisation par niveau de maturité
def categorize_maturity(score):
    if score >= 76:
        return 'Très Élevé (76-100)'
    elif score >= 51:
        return 'Moyen (51-75)'
    elif score >= 26:
        return 'Faible (26-50)'
    else:
        return 'Très Faible (0-25)'

df['maturity_level'] = df['maturity_score'].apply(categorize_maturity)

# Distribution
maturity_dist = df['maturity_level'].value_counts()
print("\n🎯 Répartition par Niveau de Maturité:")
for level in ['Très Faible (0-25)', 'Faible (26-50)', 'Moyen (51-75)', 'Très Élevé (76-100)']:
    count = maturity_dist.get(level, 0)
    pct = (count / len(df)) * 100
    print(f"  {level}: {count} cliniques ({pct:.1f}%)")


🎯 Répartition par Niveau de Maturité:
  Très Faible (0-25): 11 cliniques (39.3%)
  Faible (26-50): 15 cliniques (53.6%)
  Moyen (51-75): 2 cliniques (7.1%)
  Très Élevé (76-100): 0 cliniques (0.0%)


## 3. Visualisations

In [6]:
# Distribution des scores
fig = px.histogram(df, x='maturity_score', 
                   nbins=20,
                   title='Distribution des Scores de Maturité Digitale',
                   labels={'maturity_score': 'Score de Maturité (/100)', 'count': 'Nombre de cliniques'},
                   color_discrete_sequence=['#4ECDC4'])

fig.add_vline(x=df['maturity_score'].mean(), 
              line_dash="dash", 
              line_color="red",
              annotation_text=f"Moyenne: {df['maturity_score'].mean():.1f}")

fig.update_layout(height=500, showlegend=False)
fig.show()

In [7]:
# Répartition par catégorie
category_order = ['Très Faible (0-25)', 'Faible (26-50)', 'Moyen (51-75)', 'Très Élevé (76-100)']
colors = ['#FF6B6B', '#FFA07A', '#4ECDC4', '#45B7D1']

maturity_counts = df['maturity_level'].value_counts().reindex(category_order, fill_value=0)

fig = go.Figure(data=[go.Pie(
    labels=maturity_counts.index,
    values=maturity_counts.values,
    marker_colors=colors,
    hole=0.4,
    textinfo='label+percent',
    textposition='outside'
)])

fig.update_layout(
    title='Répartition des Cliniques par Niveau de Maturité',
    height=600
)
fig.show()

In [8]:
# Top 10 cliniques par score
top_10 = df.nlargest(10, 'maturity_score')[['clinic_name', 'location', 'maturity_score', 'maturity_level']]

fig = go.Figure(data=[go.Bar(
    x=top_10['maturity_score'],
    y=top_10['clinic_name'],
    orientation='h',
    marker=dict(
        color=top_10['maturity_score'],
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="Score")
    ),
    text=top_10['maturity_score'],
    textposition='auto'
)])

fig.update_layout(
    title='Top 10 Cliniques - Score de Maturité Digitale',
    xaxis_title='Score (/100)',
    yaxis_title='',
    height=600,
    yaxis={'categoryorder': 'total ascending'}
)
fig.show()

print("\n🏆 Top 10 Cliniques:")
print(top_10.to_string(index=False))


🏆 Top 10 Cliniques:
                      clinic_name                                                    location  maturity_score maturity_level
                            PISAM                                                      Cocody            55.0  Moyen (51-75)
      CLINIQUE MEDICALE CRYSALIDE                                         YOPOUGON TOIT-ROUGE            55.0  Moyen (51-75)
          Hôpital islamique Daloa                                    Savonnerie Route d’issia            47.5 Faible (26-50)
             Clinique hôtel Maria                                                    Yopougon            47.5 Faible (26-50)
                   Clinique Malon                                                  Port bouet            47.5 Faible (26-50)
             POLYCLINIQUE KENNEDY                                                      BOUAKÉ            45.0 Faible (26-50)
CLINIQUE MEDICALE MISERICODE DIEU                                    YOPOUGON ANCIEN BEL AIRE           

## 4. Analyse par Taille de Clinique

In [9]:
# Catégorisation par taille
df['size_category'] = pd.cut(
    df['number_of_beds'],
    bins=[0, 15, 50, float('inf')],
    labels=['Petite (<15 lits)', 'Moyenne (15-50 lits)', 'Grande (>50 lits)']
)

# Analyse par taille
size_analysis = df.groupby('size_category')['maturity_score'].agg([
    ('Nombre', 'count'),
    ('Moyenne', 'mean'),
    ('Médiane', 'median'),
    ('Min', 'min'),
    ('Max', 'max'),
    ('Écart-type', 'std')
]).round(1)

print("📏 Analyse de Maturité par Taille:")
print(size_analysis)

# Visualisation
fig = px.box(
    df,
    x='size_category',
    y='maturity_score',
    points='all',
    title='Score de Maturité par Taille de Clinique',
    labels={'size_category': 'Taille', 'maturity_score': 'Score (/100)'},
    color='size_category',
    color_discrete_sequence=['#FF6B6B', '#4ECDC4', '#45B7D1']
)
fig.update_layout(height=500, showlegend=False)
fig.show()

📏 Analyse de Maturité par Taille:
                      Nombre  Moyenne  Médiane   Min   Max  Écart-type
size_category                                                         
Petite (<15 lits)         13     21.9     25.0   0.0  47.5        16.5
Moyenne (15-50 lits)       8     34.4     37.5   0.0  47.5        14.7
Grande (>50 lits)          4     47.5     51.2  32.5  55.0        10.6


## 5. Analyse par Localisation

In [10]:
# Extraction de la ville principale
def extract_city(location):
    if pd.isna(location):
        return 'Non spécifié'
    location = str(location).upper()
    cities = ['COCODY', 'YOPOUGON', 'DALOA', 'BOUAKÉ', 'BONOUA', 'ABOBO', 'BINGERVILLE']
    for city in cities:
        if city in location:
            return city.capitalize()
    return 'Autre'

df['city'] = df['location'].apply(extract_city)

# Analyse par ville
city_analysis = df.groupby('city').agg({
    'maturity_score': ['count', 'mean', 'std']
}).round(1)
city_analysis.columns = ['Nombre', 'Score Moyen', 'Écart-type']
city_analysis = city_analysis.sort_values('Score Moyen', ascending=False)

print("🌍 Analyse de Maturité par Ville:")
print(city_analysis)

# Visualisation
fig = go.Figure(data=[go.Bar(
    x=city_analysis.index,
    y=city_analysis['Score Moyen'],
    error_y=dict(type='data', array=city_analysis['Écart-type']),
    marker_color='#4ECDC4',
    text=city_analysis['Score Moyen'].round(1),
    textposition='auto'
)])

fig.update_layout(
    title='Score Moyen de Maturité par Ville',
    xaxis_title='Ville',
    yaxis_title='Score Moyen (/100)',
    height=500
)
fig.show()

🌍 Analyse de Maturité par Ville:
              Nombre  Score Moyen  Écart-type
city                                         
Yopougon           3         48.3         6.3
Cocody             2         46.2        12.4
Daloa              3         33.3         7.6
Bonoua             2         31.2         8.8
Autre             10         29.0        16.8
Abobo              2         28.8         5.3
Bouaké             4         17.5        21.8
Bingerville        1          0.0         NaN
Non spécifié       1          0.0         NaN


## 6. Analyse Détaillée des Composantes

In [11]:
# Taux d'adoption par composante
components = {
    'Système informatisé\n(25 pts)': 'has_informatic_management_system',
    'Site web\n(20 pts)': 'uses_website',
    'Réseaux sociaux\n(20 pts)': 'uses_social_media',
    'Outils RDV\n(15 pts)': 'uses_digital_tools_for_appointments',
    'Stratégie digitale\n(10 pts)': 'has_formal_digital_strategy',
    'Équipe dédiée\n(5 pts)': 'has_dedicated_digital_team',
    'Budget dédié\n(5 pts)': 'has_dedicated_digital_budget'
}

adoption_rates = {}
for name, col in components.items():
    rate = (df[col].sum() / len(df)) * 100
    adoption_rates[name] = rate

adoption_df = pd.DataFrame.from_dict(adoption_rates, orient='index', columns=['Taux'])
adoption_df = adoption_df.sort_values('Taux', ascending=True)

# Visualisation
fig = go.Figure(go.Bar(
    x=adoption_df['Taux'],
    y=adoption_df.index,
    orientation='h',
    marker=dict(
        color=adoption_df['Taux'],
        colorscale='RdYlGn',
        showscale=True,
        colorbar=dict(title="%")
    ),
    text=[f"{val:.1f}%" for val in adoption_df['Taux']],
    textposition='auto'
))

fig.update_layout(
    title='Taux d\'Adoption par Composante de la Maturité Digitale',
    xaxis_title='Taux d\'adoption (%)',
    yaxis_title='',
    height=500
)
fig.add_vline(x=50, line_dash="dash", line_color="gray", annotation_text="50%")
fig.show()

print("\n📊 Taux d'Adoption par Composante:")
print(adoption_df.to_string())


📊 Taux d'Adoption par Composante:
                                    Taux
Site web\n(20 pts)              0.000000
Réseaux sociaux\n(20 pts)       0.000000
Budget dédié\n(5 pts)          14.285714
Équipe dédiée\n(5 pts)         21.428571
Outils RDV\n(15 pts)           30.357143
Stratégie digitale\n(10 pts)   32.142857
Système informatisé\n(25 pts)  78.571429


## 7. Corrélations et Insights

In [12]:
# Relation entre taille et maturité
fig = px.scatter(
    df,
    x='number_of_beds',
    y='maturity_score',
    size='number_of_healthcare_staff',
    color='size_category',
    hover_data=['clinic_name', 'location'],
    title='Relation entre Taille (lits) et Maturité Digitale',
    labels={'number_of_beds': 'Nombre de lits', 'maturity_score': 'Score de Maturité (/100)'},
    trendline='ols'
)
fig.update_layout(height=600)
fig.show()

# Calcul de la corrélation
correlation = df[['number_of_beds', 'maturity_score']].corr().iloc[0, 1]
print(f"\n📈 Corrélation entre nombre de lits et score de maturité: {correlation:.3f}")
if correlation > 0.5:
    print("   → Corrélation positive forte : les grandes cliniques sont plus matures")
elif correlation > 0.3:
    print("   → Corrélation positive modérée")
else:
    print("   → Corrélation faible")

ValueError: 
    Invalid element(s) received for the 'size' property of scatter.marker
        Invalid elements include: [nan]

    The 'size' property is a number and may be specified as:
      - An int or float in the interval [0, inf]
      - A tuple, list, or one-dimensional numpy array of the above

## 8. Export des Résultats

In [ ]:
# Création du rapport par clinique
report_cols = ['clinic_name', 'location', 'number_of_beds', 'size_category', 'city',
               'maturity_score', 'maturity_level',
               'has_informatic_management_system', 'uses_website', 'uses_social_media',
               'uses_digital_tools_for_appointments', 'has_formal_digital_strategy',
               'has_dedicated_digital_team', 'has_dedicated_digital_budget']

report_df = df[report_cols].copy()
report_df = report_df.sort_values('maturity_score', ascending=False)

# Renommer les colonnes pour l'export
report_df.columns = [
    'Clinique', 'Localisation', 'Lits', 'Catégorie Taille', 'Ville',
    'Score Maturité', 'Niveau',
    'Système Info', 'Site Web', 'Réseaux Sociaux',
    'Outils RDV', 'Stratégie', 'Équipe', 'Budget'
]

# Sauvegarder
import os
os.makedirs('data', exist_ok=True)
report_df.to_csv('data/scores_maturite.csv', index=False, encoding='utf-8-sig')
print("✅ Rapport sauvegardé : data/scores_maturite.csv")

# Aperçu
print("\n📋 Aperçu du rapport (Top 10):")
print(report_df.head(10).to_string(index=False))

## 9. Génération de Recommandations

In [ ]:
def generate_recommendations(row):
    """Génère des recommandations personnalisées pour une clinique"""
    recommendations = []
    priority = 1
    
    score = row['maturity_score']
    
    # Recommandations selon le score global
    if score < 50:
        recommendations.append({
            'Priorité': priority,
            'Catégorie': 'Fondation',
            'Action': 'Évaluation complète des besoins',
            'Budget': '500K-1M FCFA',
            'Délai': '1-2 mois'
        })
        priority += 1
    
    # Recommandations spécifiques par composante
    if row['has_informatic_management_system'] < 1:
        recommendations.append({
            'Priorité': priority,
            'Catégorie': 'Système',
            'Action': 'Implémenter système de gestion informatisé (SIH)',
            'Budget': '1M-3M FCFA',
            'Délai': '2-3 mois'
        })
        priority += 1
    
    if row['uses_website'] < 1:
        recommendations.append({
            'Priorité': priority,
            'Catégorie': 'Web',
            'Action': 'Créer site web professionnel',
            'Budget': '300K-800K FCFA',
            'Délai': '1-2 mois'
        })
        priority += 1
    
    if row['uses_social_media'] < 1:
        recommendations.append({
            'Priorité': priority,
            'Catégorie': 'Social',
            'Action': 'Établir présence réseaux sociaux (Facebook, Instagram)',
            'Budget': '100K-300K FCFA/mois',
            'Délai': '1 mois'
        })
        priority += 1
    
    if row['uses_digital_tools_for_appointments'] < 1:
        recommendations.append({
            'Priorité': priority,
            'Catégorie': 'Automatisation',
            'Action': 'Mettre en place système de prise de RDV en ligne',
            'Budget': '200K-500K FCFA',
            'Délai': '1 mois'
        })
        priority += 1
    
    if row['has_formal_digital_strategy'] < 1:
        recommendations.append({
            'Priorité': priority,
            'Catégorie': 'Stratégie',
            'Action': 'Développer stratégie de communication digitale',
            'Budget': '500K-1M FCFA',
            'Délai': '2-3 mois'
        })
        priority += 1
    
    if row['has_dedicated_digital_team'] < 1 and score > 40:
        recommendations.append({
            'Priorité': priority,
            'Catégorie': 'Ressources',
            'Action': 'Recruter ou former une personne dédiée au digital',
            'Budget': 'Salaire mensuel',
            'Délai': '2-3 mois'
        })
        priority += 1
    
    if row['has_dedicated_digital_budget'] < 1 and score > 30:
        recommendations.append({
            'Priorité': priority,
            'Catégorie': 'Budget',
            'Action': 'Allouer budget mensuel pour activités digitales',
            'Budget': '5-10% budget marketing',
            'Délai': 'Immédiat'
        })
        priority += 1
    
    # Si déjà mature, recommandations d'optimisation
    if score >= 70:
        recommendations.append({
            'Priorité': priority,
            'Catégorie': 'Innovation',
            'Action': 'Explorer solutions IA pour optimisation processus',
            'Budget': 'Variable',
            'Délai': '3-6 mois'
        })
    
    return recommendations

# Générer recommandations pour toutes les cliniques
all_recommendations = []
for idx, row in df.iterrows():
    recs = generate_recommendations(row)
    for rec in recs:
        rec['Clinique'] = row['clinic_name']
        rec['Score Actuel'] = row['maturity_score']
        all_recommendations.append(rec)

reco_df = pd.DataFrame(all_recommendations)
reco_df = reco_df[['Clinique', 'Score Actuel', 'Priorité', 'Catégorie', 'Action', 'Budget', 'Délai']]

# Sauvegarder
os.makedirs('reports', exist_ok=True)
reco_df.to_csv('reports/recommandations_par_clinique.csv', index=False, encoding='utf-8-sig')
print("✅ Recommandations sauvegardées : reports/recommandations_par_clinique.csv")

# Aperçu pour une clinique
sample_clinic = df.iloc[0]['clinic_name']
sample_recs = reco_df[reco_df['Clinique'] == sample_clinic]
print(f"\n📋 Exemple de recommandations pour {sample_clinic}:")
print(sample_recs.to_string(index=False))

## 10. Conclusions et Insights Clés

In [ ]:
print("="*80)
print("🎯 CONCLUSIONS ET INSIGHTS CLÉS")
print("="*80)

print(f"\n1️⃣  SCORE GLOBAL")
print(f"   • Score moyen : {df['maturity_score'].mean():.1f}/100")
print(f"   • {(df['maturity_score'] >= 51).sum()} cliniques ({(df['maturity_score'] >= 51).sum()/len(df)*100:.0f}%) ont un score moyen ou élevé")
print(f"   • {(df['maturity_score'] < 51).sum()} cliniques ({(df['maturity_score'] < 51).sum()/len(df)*100:.0f}%) ont un score faible")

print(f"\n2️⃣  DISPARITÉS PAR TAILLE")
for size in ['Petite (<15 lits)', 'Moyenne (15-50 lits)', 'Grande (>50 lits)']:
    if size in df['size_category'].values:
        avg = df[df['size_category'] == size]['maturity_score'].mean()
        count = df[df['size_category'] == size].shape[0]
        print(f"   • {size}: {avg:.1f}/100 ({count} cliniques)")

print(f"\n3️⃣  COMPOSANTES LES PLUS FAIBLES")
weak_components = adoption_df.nsmallest(3, 'Taux')
for comp, row in weak_components.iterrows():
    print(f"   • {comp.split('(')[0].strip()}: {row['Taux']:.0f}% d'adoption")

print(f"\n4️⃣  OPPORTUNITÉS PRINCIPALES")
print(f"   • {(df['uses_website'] < 1).sum()} cliniques sans site web → Opportunité création web")
print(f"   • {(df['has_formal_digital_strategy'] < 1).sum()} cliniques sans stratégie → Besoin consulting")
print(f"   • {(df['has_dedicated_digital_team'] < 1).sum()} cliniques sans équipe dédiée → Besoin formation/recrutement")

print(f"\n5️⃣  RECOMMANDATIONS STRATÉGIQUES")
print("   ✅ Développer une offre d'accompagnement différenciée par niveau de maturité")
print("   ✅ Prioriser les petites cliniques pour maximiser l'impact")
print("   ✅ Créer des packages 'Quick Win' pour composantes faibles (site web, social media)")
print("   ✅ Proposer des formations collectives pour réduire les coûts")
print("   ✅ Mettre en place un système de mentoring (cliniques avancées → débutantes)")

print("\n" + "="*80)
print("✅ ANALYSE TERMINÉE AVEC SUCCÈS")
print("="*80)